In [1]:
import requests
import csv

def fetch_data_from_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data from API. Status code: {response.status_code}")
        return None

def write_data_to_csv(data, filename):
    if data:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(data)
        print(f"Data has been written to {filename}")
    else:
        print("No data to write to CSV.")

def main():
    api_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME&for=county:*"
    data = fetch_data_from_api(api_url)
    if data:
        write_data_to_csv(data, "FIPS.csv")

if __name__ == "__main__":
    main()


Data has been written to FIPS.csv


In [2]:
import pandas as pd

# Load the dataset
file_path = '/content/FIPS.csv'
df = pd.read_csv(file_path)


In [3]:
# Splitting 'NAME' column into 'county_name' and 'state_name'
df[['county_name', 'state_name']] = df['NAME'].str.split(', ', expand=True)

# Dropping the original 'NAME' column
df.drop(columns=['NAME'], inplace=True)


# Remove 'County' from 'county_name' column
df['county_name'] = df['county_name'].str.replace(' County', '')


# Dictionary mapping full state names to abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Add new column 'state_abbreviation' based on state_name
df['state_abbrev'] = df['state_name'].map(state_abbreviations)

In [ ]:
df.head()

,state,county,county_name,state_name,state_abbrev
0,5,131,Sebastian,Arkansas,AR
1,5,133,Sevier,Arkansas,AR
2,5,135,Sharp,Arkansas,AR
3,5,137,Stone,Arkansas,AR
4,5,139,Union,Arkansas,AR


In [4]:
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO
from geopy.geocoders import Nominatim
import os, re
import numpy as np


# Load the dataset
file_path = '/content/EDDLayer2_attributes.xls'
data = pd.read_excel(file_path)



In [ ]:
data.head()

In [5]:
# Function to find state_fips and county_fips
def find_fips(row):
    state_fips = None
    county_fips = []

    # Split counties and iterate over them
    counties = row['Counties'].split(', ')
    for county in counties:
        # Remove "City of" and "Towns of" prefixes
        county = county.replace("City of ", "").replace("Towns of ", "").strip()

        # Find corresponding state_name and county_name
        match = df[(df['state_abbrev'] == row['State']) & (df['county_name'] == county)]
        if not match.empty:
            # Extract state_fips and county_fips
            state_fips = match['state'].iloc[0]  # Assuming 'state' column contains state FIPS code
            county_fips.append(match['county'].iloc[0])  # Assuming 'county' column contains county FIPS code

    return pd.Series([state_fips, county_fips])

# Apply the function to update the dataframe
data[['state_fips', 'county_fips']] = data.apply(find_fips, axis=1)

# Explode the county_fips column
expanded_data = data.explode('county_fips')

# Join back with the original dataframe to get corresponding county names
expanded_data = expanded_data.merge(df, left_on=['state_fips', 'county_fips'], right_on=['state', 'county'], how='left')

# Drop unnecessary columns
expanded_data.drop(['state', 'county', 'state_name', 'state_abbrev', 'Counties', 'OBJECTID_12' , 'EDD_Name' ], axis=1, inplace=True)


In [6]:
expanded_data.head()

,EDDname,Abbrev,State,City,EDARegion,ExDir,Contact,Phone,Website,CEDS_Link,Performance_Period,Shape_Length,Shape_Area,state_fips,county_fips,county_name
0,Central Massachusetts Regional Planning Commis...,CMRPC,MA,Worcester,Philadelphia,"Janet Pierce, Executive Director",jpierce@cmrpc.org,508-459-3319,https://www.cmrpc.org/,http://www.cmrpc.org/sites/default/files/Docum...,2018 - 2023,362569.129663,4.540662e+09,NaN,NaN,NaN
1,Commonwealth Regional Council,CRC,VA,Farmville,Philadelphia,"Melody Foster, Executive Director",mfoster@virginiasheartland.org,434-392-6104,https://www.heartlandeconomy.com/,https://www.heartlandeconomy.com/_files/ugd/1b...,2019 - 2024,849016.659357,9.029085e+09,51.0,7,Amelia
2,Commonwealth Regional Council,CRC,VA,Farmville,Philadelphia,"Melody Foster, Executive Director",mfoster@virginiasheartland.org,434-392-6104,https://www.heartlandeconomy.com/,https://www.heartlandeconomy.com/_files/ugd/1b...,2019 - 2024,849016.659357,9.029085e+09,51.0,29,Buckingham
3,Commonwealth Regional Council,CRC,VA,Farmville,Philadelphia,"Melody Foster, Executive Director",mfoster@virginiasheartland.org,434-392-6104,https://www.heartlandeconomy.com/,https://www.heartlandeconomy.com/_files/ugd/1b...,2019 - 2024,849016.659357,9.029085e+09,51.0,37,Charlotte
4,Commonwealth Regional Council,CRC,VA,Farmville,Philadelphia,"Melody Foster, Executive Director",mfoster@virginiasheartland.org,434-392-6104,https://www.heartlandeconomy.com/,https://www.heartlandeconomy.com/_files/ugd/1b...,2019 - 2024,849016.659357,9.029085e+09,51.0,49,Cumberland


In [8]:
# Save the updated dataset to a new Excel file
updated_file_path = '/content/updated_EDDLayer2_attributes.csv'
expanded_data.to_csv(updated_file_path, index=False)

print(f"Updated file saved to {updated_file_path}")

Updated file saved to /content/updated_EDDLayer2_attributes.csv
